In [1]:
import os
import warnings
import tensorflow as tf 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.callbacks import Callback
import numpy as np

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

2. Define the model:
Create a simple neural network model with a Flatten layer followed by two Dense layers.

In [2]:
# Step 2: Define the Model

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])

3. Define Loss Function and Optimizer:
Use Sparse Categorical Crossentropy for the loss function.
Use the Adam optimizer.

In [8]:
loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimize=tf.keras.optimizers.Adam()
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training

## 4. Implement the Custom Training Loop:
Iterate over the dataset for a specified number of epochs.
Compute the loss and apply gradients to update the model's weights.

In [9]:
# Step 4: Implement the Custom Training Loop

epochs = 2
# train_dataset = train_dataset.repeat(epochs)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)  # Forward pass
            loss_value = loss_fn(y_batch_train, logits)  # Compute loss

        # Compute gradients and update weights
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimize.apply_gradients(zip(grads, model.trainable_weights))
        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)
        # Logging the loss every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()}  Accuracy = {accuracy_metric.result().numpy()} ')
# Reset the metric at the end of each epoch
accuracy_metric.reset_state()
            

Start of epoch 1
Epoch 1 Step 0: Loss = 0.04315980523824692  Accuracy = 1.0 
Epoch 1 Step 200: Loss = 0.07757634669542313  Accuracy = 0.9751243591308594 
Epoch 1 Step 400: Loss = 0.05379654839634895  Accuracy = 0.9724127054214478 
Epoch 1 Step 600: Loss = 0.032775189727544785  Accuracy = 0.9737936854362488 
Epoch 1 Step 800: Loss = 0.04144629091024399  Accuracy = 0.9739778637886047 
Epoch 1 Step 1000: Loss = 0.17775724828243256  Accuracy = 0.9746191501617432 
Epoch 1 Step 1200: Loss = 0.07076407968997955  Accuracy = 0.9746565222740173 
Epoch 1 Step 1400: Loss = 0.10073287785053253  Accuracy = 0.9748393893241882 
Epoch 1 Step 1600: Loss = 0.10931351035833359  Accuracy = 0.9745861887931824 
Epoch 1 Step 1800: Loss = 0.04842286556959152  Accuracy = 0.9750832915306091 
Start of epoch 2
Epoch 2 Step 0: Loss = 0.015033794566988945  Accuracy = 0.9754464030265808 
Epoch 2 Step 200: Loss = 0.07736096531152725  Accuracy = 0.9761560559272766 
Epoch 2 Step 400: Loss = 0.044564854353666306  Accurac

### Custom Callback for Advanced Logging

In [11]:
from tensorflow.keras.callbacks import Callback

# Step 4: Implement the Custom Callback 
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        print(f'End of epoch {epoch + 1}, loss: {logs.get("loss")}, accuracy: {logs.get("accuracy")}')

In [13]:
# Step 5: Implement the Custom Training Loop with Custom Callback

epochs = 2
custom_callback = CustomCallback()  # Initialize the custom callback

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)
        
        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimize.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')
    
    # Call the custom callback at the end of each epoch
    custom_callback.on_epoch_end(epoch, logs={'loss': loss_value.numpy(), 'accuracy': accuracy_metric.result().numpy()})
    
    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()  # Use reset_state() instead of reset_states()

Start of epoch 1
Epoch 1 Step 0: Loss = 0.014878474175930023 Accuracy = 1.0
Epoch 1 Step 200: Loss = 0.06551411747932434 Accuracy = 0.9872512221336365
Epoch 1 Step 400: Loss = 0.031689975410699844 Accuracy = 0.9865959882736206
Epoch 1 Step 600: Loss = 0.03019825741648674 Accuracy = 0.986896812915802
Epoch 1 Step 800: Loss = 0.016490695998072624 Accuracy = 0.9868913888931274
Epoch 1 Step 1000: Loss = 0.08944132924079895 Accuracy = 0.9874188303947449
Epoch 1 Step 1200: Loss = 0.036088816821575165 Accuracy = 0.9875364303588867
Epoch 1 Step 1400: Loss = 0.04080403968691826 Accuracy = 0.9874197244644165
Epoch 1 Step 1600: Loss = 0.05401533097028732 Accuracy = 0.9873126149177551
Epoch 1 Step 1800: Loss = 0.023965956643223763 Accuracy = 0.9875763654708862
End of epoch 1, loss: 0.028269894421100616, accuracy: 0.9877166748046875
Start of epoch 2
Epoch 2 Step 0: Loss = 0.00859090220183134 Accuracy = 1.0
Epoch 2 Step 200: Loss = 0.03307466208934784 Accuracy = 0.9902052283287048
Epoch 2 Step 400: 